In [0]:
%run "./config/v5/configurations-all-line"

In [0]:
%run "./config/v5/common"

data_source_id,schema_version,schema_ddl,logic_mapping
b31d684c-d84b-44fa-9873-47c561542df9,6,"`Line` STRING, `Station` STRING, `Part Number` STRING, `Database Code` STRING, `Serial Number` STRING, `Time` STRING, `Gun` STRING, `Job` STRING, `Pass` STRING, `Torque` STRING, `Ang (deg.)` STRING","{""line"":[""Line""], ""station_config"":[""Line"", ""Station""], ""sensor_config"":[""Line"", ""Station"", ""Gun"", ""Job""], ""part_number"": [""Part_Number""], ""serial_number"": [""Serial_Number""], ""measurement"":[""Torque"", ""Ang__deg__""], ""measured_time"":""Time""}"
d21ae4b8-51d8-4990-b9da-91c5cfc67927,7,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""part_number"": [""part_number""], ""serial_number"": [""serial_number""], ""measurement"":[""data_value""], ""measured_time"":""created""}"
86494c9c-143c-2e85-3b50-2021de22c403,9,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""part_number"": [""part_number""], ""serial_number"": [""serial_number""], ""measurement"":[""data_value""], ""measured_time"":""created""}"
2665e070-cbde-ac9f-d6d0-228d82bc75ac,8,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""

data_source_id,client,location,line,source_type,folder_location
b31d684c-d84b-44fa-9873-47c561542df9,Borg Warner,XYZ,L14,csv,server://path
d21ae4b8-51d8-4990-b9da-91c5cfc67927,Dana,Birmingham,ASC1,csv,server://path
86494c9c-143c-2e85-3b50-2021de22c403,Dana,Dry Ridge,700_line,csv,server://path
2665e070-cbde-ac9f-d6d0-228d82bc75ac,Dana,Columbia,Loop_1,csv,server://path


In [0]:
#**********************************************************************************************************************
# RESET STREAMING (for testing and dev only)
# use this code block to clear streaming checkpoints and output folders 
#**********************************************************************************************************************

dbutils.fs.rm(checkpointPath, True)
dbutils.fs.rm(pivotRootDir, True)
dbutils.fs.rm(cubeRunRootDir, True)


#**********************************************************************************************************************
# RESET PIVOT SCHEMA TABLE (for testing and dev only)
# use this code block to create an empty pivot schema table 
#**********************************************************************************************************************

pivot_schema_df = spark.createDataFrame(data=[],schema=T._parse_datatype_string(pivot_schema_table_ddl))
pivot_schema_df.registerTempTable("pivot_schema_table")
pivot_schema_df.show()

+------------+------------------+-------------+-----------+----------+
dataSourceId|pivotSchemaVersion|pivotIndexDdl|featureList|updated_on|
+------------+------------------+-------------+-----------+----------+
+------------+------------------+-------------+-----------+----------+

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark import Row
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.types import *
import json

#**********************************************************************************************************************
# HOUSE KEEPING
#**********************************************************************************************************************

# set the basic spark configurations
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

# load configuration and other variables through Notebook workflow
#dbutils.notebook.run("./configurations", 60)


#**********************************************************************************************************************
# DATA INGESTION: read stream from Kafka topic and apply ingress data schema
#**********************************************************************************************************************

input_df = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", conf['bootstrap.servers'])
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(conf['sasl.username'], conf['sasl.password']))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", conf['topic'])
  .option("startingOffsets", conf['startingOffsets'])
  .option("failOnDataLoss", "false")
  .option("maxOffsetsPerTrigger", "1")
  .load()
  .select(col("value").cast("STRING"), col("key").cast("STRING")) 
  .select(from_json("value", ingress_schema).alias("ingress_json"))
)


#**********************************************************************************************************************
# TRANSFORM STEP 1/5: extract data source id, schema version, and measurement dataframe from ingested Kafka record.
# The column headers of the measurement dataframe are cleaned to remove illegal characters.
#**********************************************************************************************************************

def replace_special_char(column_name):
  """ replace all non-alphanumeric characters with an underscore. """
  return regexp_replace(column_name, r'[^a-zA-Z0-9]', '_')

formatted_df = (input_df
                .select(col("ingress_json.dataSourceId").alias("dataSourceId"), 
                        col("ingress_json.schemaVersion").alias("schemaVersion"), explode("ingress_json.rows").alias("records"))
                .withColumn("headers", transform(f.map_keys("records"), replace_special_char))
                .withColumn("data", map_values("records"))
               .withColumn("mapped_fields",  map_from_arrays("headers", "data"))
               .select("dataSourceId", "schemaVersion", "mapped_fields")
              )


#**********************************************************************************************************************
# TRANSFORM STEP 2/5: apply business logic mapping to incoming data columns
# NOTE: the business-mapping config json string is read from the business-schema table and applied in realtime to
# the data records. Records with different data source id and schema version will have different schemas applied.
#**********************************************************************************************************************

def mapLogicalFields(fields, logic_mapping):
  """
  udf to map data fields according to business logic mapping config
  
  Parameters
    ----------
    fields : array<map>
        An array of maps with column header as keys and data as values. One array corresponds to one row of data 
    logic_mapping : string
        The json string containing business mapping config

    Returns
    -------
    logic_mapped_dict : dict
        A dict with the logic entites (line, station, sensor, part number, serial number, timestamp) as keys
        and data mapped from the incoming data records as values
  """
  
  mappings = json.loads(logic_mapping)
  ret, measurement_dict = {}, {}
  for attr in mappings:
    if attr=='measured_time':
      ret[config_col_mapping[attr]]=fields[mappings[attr]]
    elif attr=='measurement':
      for m in mappings[attr]:
        measurement_dict[m]=fields[m]
    else:
      ret[config_col_mapping[attr]] = '_'.join([fields[c] for c in mappings[attr]])
  ret['measurements']=measurement_dict
  return ret

# register the udf
logicMapUDF = udf(lambda x, y: mapLogicalFields(x, y), mapped_schema)


mapped_df = (formatted_df
             .join(business_schema_df, 
                   (formatted_df["dataSourceId"]==business_schema_df["data_source_id"]) & 
                   (formatted_df["schemaVersion"]==business_schema_df["schema_version"])    # EXCEPTION HANDLING for unknown source/schemaVersion
                  )
             .withColumn("logic_mapped", logicMapUDF(col("mapped_fields"), col("logic_mapping")))
             .select("dataSourceId", "schemaVersion", "logic_mapped")
             .filter("logic_mapped.serial_number!=''")     # DISCUSSION: how to handle records with missing serial numbers
            )

display(mapped_df)

dataSourceId,schemaVersion,logic_mapped
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Core, 12639700098, 0542106241600011, L14_Core_03, L14_Core_03_1_2, 2021-03-15 12:58:24.953, Map(Torque -> 181.41, Ang__deg__ -> 90))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106102700008, L14_Final_03, L14_Final_03_1_2, 2021-03-15 12:58:31.347, Map(Torque -> 10.47, Ang__deg__ -> 43))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106411600017, L14_Final_04, L14_Final_04_1_2, 2021-03-15 12:58:35.550, Map(Torque -> 17.01, Ang__deg__ -> 978))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106411600017, L14_Final_04, L14_Final_04_2_1, 2021-03-15 12:58:52.950, Map(Torque -> 13.63, Ang__deg__ -> 40))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106411600017, L14_Final_04, L14_Final_04_2_1, 2021-03-15 12:58:59.017, Map(Torque -> 13.65, Ang__deg__ -> 60))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106411600017, L14_Final_04, L14_Final_04_2_1, 2021-03-15 12:59:04.643, Map(Torque -> 13.63, Ang__deg__ -> 52))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 0542106411600017, L14_Final_04, L14_Final_04_2_1, 2021-03-15 12:59:10.457, Map(Torque -> 13.68, Ang__deg__ -> 47))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Core, 12639700098, 0542106071900004, L14_Core_02, L14_Core_02_1_1, 2021-03-15 12:59:33.603, Map(Torque -> 13.71, Ang__deg__ -> 14))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Final, 12639700098, 14A074210130, L14_Final_06, L14_Final_06_1_2, 2021-03-15 12:59:47.267, Map(Torque -> 17.01, Ang__deg__ -> 1006))"
b31d684c-d84b-44fa-9873-47c561542df9,6,"List(L14_Core, 12639700098, 0542106223500010, L14_Core_03, L14_Core_03_1_2, 2021-03-15 13:00:12.913, Map(Torque -> 170.02, Ang__deg__ -> 90))"


In [0]:
#**********************************************************************************************************************
# TRANSFORM STEP 3/5: replace sensor name with uuid, then return a dataframe with business entities as column headers, 
# a list of feature names, and a kev-value list of feature-name and data
#**********************************************************************************************************************
uuid_df = (mapped_df
           .join(sensorDf.select("sensor", "sensor_uuid"), mapped_df["logic_mapped.sensor"]==sensorDf["sensor"], "left")
           .withColumn("feature_names", transform(map_keys("logic_mapped.measurements"), lambda x: concat(col('sensor_uuid'), lit('_'), x)))
           .withColumn("feature_json", transform(arrays_zip("feature_names", map_values("logic_mapped.measurements")),
                                                 lambda x: concat(lit('"'), x["feature_names"], lit('":"'), x['1'], lit('"'))
                                                ))
           .select("dataSourceId", "schemaVersion",
                   col("logic_mapped.line").alias('line'), col("logic_mapped.part_number").alias('part_number'), 
                   col("logic_mapped.serial_number").alias('serial_number'), col("logic_mapped.station").alias('station'), 
                   col("logic_mapped.timestamp").alias('timestamp'), "feature_json", "feature_names"
                  )
          )


#**********************************************************************************************************************
# TRANSFORM STEP 4/5: collapse rows with same timestamp for each serial number into a single row, and create a
# 'measurement_date' column to be used as partition column
#**********************************************************************************************************************
collapsed_df = (uuid_df
                 .groupBy("dataSourceId", "schemaVersion", "line", "part_number","serial_number", "timestamp")
                 .agg(collect_set('station').alias("station_list"), 
                      concat(lit("{"), concat_ws(", ", array_distinct(flatten(collect_list("feature_json")))), lit("}")).alias("feature_json"),
                      array_distinct(flatten(collect_list("feature_names"))).alias("feature_names")
                     )
                 .withColumn("measurement_date", to_date("timestamp"))
                )


#**********************************************************************************************************************
# TRANSFORM STEP 5/5: join with source_meta_data_table to get full name of the line, which will decide which cube
# the transformed data is dropped into
#**********************************************************************************************************************
pivot_prep_df = (collapsed_df
                 .join(
                   data_source_meta_df.select("data_source_id", concat_ws("_", "client", "location", "line").alias("line_full_name")), 
                   collapsed_df["dataSourceId"]==data_source_meta_df["data_source_id"])
                 .withColumn("line_full_name", replace_special_char("line_full_name"))
                 .drop("data_source_id")
                )


#display(pivot_prep_df)

In [0]:
### helper functions needed to dynamically generate and update pivot table schema

def getCurrPivotSchemaLocal(dataSourceId):
  """
  returns the current features list from the pivot-schama-table and the computed next pivot schema version number 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The data source schema version of the processed records

    Returns
    -------
    features_list : set
        A list of current features
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  res = spark.sql(f'select featureList, pivotSchemaVersion from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}")').first()
  if res is None:
    return set(), 1
  return set(res[0].split(',')), res[1]+1


### TO BE IMPLEMENTED
def getCurrPivotSchemaPostgres(dataSourceId, schemaVersion):
  """ returns the current features list from a postgreSQL table and the computed next pivot schema version number """
  pass


def updatePivotSchemaLocal(data_source_id, pivot_idx_ddl, pivot_schema_version, feature_list):
  """
  updates the features list and pivot-schema version in the pivot-schama-table 
  and returns the next pivot-schema version number 
  
  Parameters
    ----------
    data_source_id : string
        The ID string of the data source 
    schema_version : string
        The data source schema version of the processed records
    pivot_idx_ddl : string
        The ddl string of index columns in the pivot-schema-table
    pivot_schema_version : int
        Pivot-schema version number of the new schema to be inserted
    feature_list : set
        A list of current features in the new schema

    Returns
    -------
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  feature_list = ','.join(feature_list)
  pivot_schema_df = (spark.sql("select * from pivot_schema_table")
                   .union(spark.createDataFrame([(data_source_id, pivot_schema_version, pivot_idx_ddl, 
                                                  feature_list, datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:23])]))
                  )
  pivot_schema_df.registerTempTable("pivot_schema_table")
  return pivot_schema_version+1


### TO BE IMPLEMENTED
def update_pivot_schema_postgres(dataSourceId, schemaVersion):
  """
  updates the features list and pivot-schema version in the postgreSQL table 
  and returns the next pivot-schema version number 
  """
  pass


def getDataSourceAndSchemaVersion(df):
  """
  returns the first data source id and schema version from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'dataSourceId' and 'schemaVersion' columns

    Returns
    -------
    dataSourceId : string
        Data source ID of the data record
    SchemaVersion : int 
        Schema version of the data record
  """
  res = df.select("dataSourceId", "schemaVersion").first()
  if res is None:
    return "", ""
  return res[0], res[1]


def getFeatureListFromDF(df):
  """
  returns the list of unique feature names from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'feature_names' column

    Returns
    -------
    features_list : list
        A list of unique feature names
  """
  
  res = df.agg(array_distinct(flatten(collect_list("feature_names")))).first()
  if res is None:
    return []
  return res[0]


def getFeatureListLocal(dataSourceId):
  """
  returns the current features list from the pivot-schama-table 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    features_list : set
        A list of current features
  """
  res = spark.sql(f'select featureList from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}")').first()
  if res is None:
    return set()
  return set(res[0].split(','))


def initialize_delta_sink(line):
  """ create an empty delta table in the cube directory for a given line """
  (spark
 .createDataFrame([], schema=T._parse_datatype_string(pivot_idx_ddl))
 .write
 .option("mergeSchema", "true")
 .partitionBy("measurement_date") 
 .format("delta")
 .mode("append")
 .save(pivotRootDir + f"{line}/"))
  
  
def load_features_for_line(data_source_id, schema_version):
  """
  returns the current features list, next pivot schema version, and current features ddl string for
  given data_source_id and schema_version
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    curr_features_list : set
        A list of current features
    next_pivot_schema_version : int
        The next pivot schema version
    curr_feature_ddl : string
        A ddl string of the current features
  """
  curr_feature_list, next_pivot_schema_version = getCurrPivotSchemaLocal(data_source_id)
  curr_feature_ddl = "`" + '` STRING, `'.join(sorted(curr_feature_list)) + "` STRING"
  return curr_feature_list, next_pivot_schema_version, curr_feature_ddl


def process_batch_for_line(line, line_df, data_source_id, schema_version):
  """
  Process batch dataframe for a given line, create a new version of pivot schema if new features are seen, 
  and updates the delta table sink by performing 'upserts' using ("serial_number", "part_number", 
  "timestamp", "station_list") as keys.
  
  Parameters
    ----------
    line : string
        Full name of the assembly line
    line_df : dataframe
        The batch dataframe for the given line
    data_source_id : string
        The ID string of the data source 
    schema_version : string
        The schema version of the data source
  """
  
  batch_feature_list = set(getFeatureListFromDF(line_df))
  
  if not batch_feature_list.issubset(lines_feature_dict[line]["curr_feature_list"]):
    lines_feature_dict[line]["prev_feature_list"] = lines_feature_dict[line]["curr_feature_list"]
    lines_feature_dict[line]["curr_feature_list"] = batch_feature_list.union(lines_feature_dict[line]["curr_feature_list"])
    lines_feature_dict[line]["next_pivot_schema_version"] = updatePivotSchemaLocal(data_source_id,  
                                                                                   pivot_idx_ddl, 
                                                                                   lines_feature_dict[line]["next_pivot_schema_version"],
                                                                                   lines_feature_dict[line]["curr_feature_list"]
                                                                                  )
    lines_feature_dict[line]["curr_feature_ddl"] = "`" + '` STRING, `'.join(sorted(lines_feature_dict[line]["curr_feature_list"])) + "` STRING"
    
  # the target pivot table
  pivot_out = DeltaTable.forPath(spark, pivotRootDir + f"{line}/")

  # apply current schema to the batch dataframe
  schema_df = (line_df
            .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
            .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
           )

  # rows in the pivot table needing to be updated
  if len(pivot_out.toDF().columns) > len(pivot_idx_ddl.split(',')):
    rows_to_update = (schema_df
                      .join(
                        pivot_out.toDF().alias("pivot_tab"), 
                        ["serial_number", "part_number", "timestamp", "station_list", "measurement_date"]
                      )
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", "pivot_tab.*") 
                      .withColumn("feature_json", 
                                  concat(lit('{'), 
                                         concat_ws(',', 
                                                   *[concat(lit('"'), lit(c), lit('":"'), col(c), lit('"')) 
                                                     for c in lines_feature_dict[line]["prev_feature_list"]]
                                                  ),
                                         lit('}')
                                        )
                                 )
                      .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
                     )

    # prepare the updates dataframe
    updates_df = (schema_df
                 .union(rows_to_update)
                 .groupBy("serial_number", "part_number", "timestamp", "station_list", "measurement_date")
                 .agg(*[f.max(c).alias(c) for c in lines_feature_dict[line]["curr_feature_list"]])
                )
  else:
    updates_df = schema_df

  # upsert new data into Delta table
  match_on_key = """s.part_number=u.part_number and s.serial_number=u.serial_number and s.timestamp=u.timestamp and s.station_list=u.station_list""" 

  (pivot_out
   .alias("s")
   .merge(updates_df.alias("u"), match_on_key)
   .whenMatchedUpdateAll()
   .whenNotMatchedInsertAll()
   .execute()
  )

In [0]:
#**********************************************************************************************************************
# WRITE PIVOTTED RESULTS TO DELTA TABLE (THE CUBE)
# The pivot table is dynamically populated and updated with transformed streaming data
#**********************************************************************************************************************

lines_processed, lines_feature_dict = [], {}

def updateSchemaAndDelta(df, batch_id):
  """
  Process the batch dataframe. For each line in the dataframe, update the corresponding delta table.
  
  Parameters
    ----------
    df : dataframe
        The batch dataframe 
    batch_id : string
        The batch ID
  """
  global lines_processed, lines_feature_dict
  
  lines = df.select("line_full_name").distinct().collect()
  for a_row in lines:
    line = a_row["line_full_name"]
    line_df = df.filter(col("line_full_name")==line)
    data_source_id, schema_version =  getDataSourceAndSchemaVersion(line_df)   ## ADD HANDLER for line with multiple schema versions
    
    if not line in lines_processed:
      initialize_delta_sink(line)
      
      temp_dict = {}
      temp_dict["curr_feature_list"], \
      temp_dict["next_pivot_schema_version"], \
      temp_dict["curr_feature_ddl"] = load_features_for_line(data_source_id, schema_version)
      
      lines_feature_dict[line] = temp_dict
      
      lines_processed.append(line)
    
    process_batch_for_line(line, line_df, data_source_id, schema_version)


savePivot = (pivot_prep_df
             .writeStream
             .queryName("kafka_kv_to_pivot_delta_lake")
             .trigger(processingTime="1 seconds")
             .format("delta")
             .option("checkpointLocation", checkpointPath)
                 .option("mergeSchema", "true")
                 .foreachBatch(updateSchemaAndDelta)
             .outputMode("update")
             .start()
            )

#display(input_df)

In [0]:
display(spark.sql("select * from pivot_schema_table"))

dataSourceId pivotSchemaVersion pivotIndexDdl featureList updated_on 2665e070-cbde-ac9f-d6d0-228d82bc75ac 1 `part_number` STRING, `serial_number` STRING, `timestamp` STRING, `station_list` ARRAY , `measurement_date` DATE 6898c2e3-c7d2-47c5-9a7b-adeca7d82649_data_value,fbdd41a3-c4b5-4f0e-be1e-b9e4cbe1b584_data_value,82f6e3be-a270-4937-8b4a-61623599129a_data_value,c71a2cd6-807e-4a0a-a1aa-c4e9417c38ce_data_value,a673d084-c628-45d6-a265-4c92f8fc0ae3_data_value,b42c3e3a-e0a9-421c-a80b-b908ac97fcba_data_value,e25fc6e0-d1ad-4ef9-8bd2-60630dd3c066_data_value,bc33f224-7380-4ab4-85a0-16c33cc0fcd9_data_value,00447cdb-4a6c-4a44-86a2-c8dd46a9e001_data_value,593db757-31b2-4aef-ac82-4d6b44ee17e7_data_value,1db8614a-d2f3-4f95-bfc4-590ff97531de_data_value,6b16384e-4099-40d4-840e-57c55c81a3c4_data_value,17d8983d-0896-4ec7-bc5d-a181c04a781e_data_value,a9e22ccb-0fe7-410f-93fb-227afa385c5c_data_value,0368cb03-92d9-482d-88c5-59a4ac8c5094_data_value,22c26be2-213e-46a2-8d47-64de603e3573_data_value,c2238489-50ad-41f0-b9eb-b078b707ee01_data_value,29bd9e48-5b56-46bd-9636-32d2d9bb236e_data_value,d58be0ea-da3b-4992-b44d-ed3646647e71_data_value,19a2495f-1de2-4a94-93cf-f80bbfcf5089_data_value,f4ef792e-d845-4677-9cad-17ada5c6accb_data_value,412457e2-07c0-46a1-a10b-a25aa575b91a_data_value,db4e6273-d023-4354-bce8-21aea2a466e4_data_value,29835e53-4e33-4222-b03c-924b2e9ea185_data_value,497af229-7cf7-4470-bd4b-86a116207077_data_value,78583988-8b9e-4e44-8f58-923a44571440_data_value,2b0d14f5-105a-45ee-85fc-f51ec1dd15f5_data_value,f03089c9-5228-4e4d-8ac9-6dbfe1e2eebc_data_value,653a1609-552d-479b-a14e-62ba56aab8e0_data_value,e15acf73-f4f1-4670-8f43-2afbe33f8975_data_value,ec39a4c2-84d9-4de9-a213-12e691862e5a_data_value,b3dc41e7-3d49-4806-a967-e038d2824304_data_value,6fdeae3c-47f8-4920-bd6e-88e8dfcd3d00_data_value,00179971-75bd-44ae-bba9-9f6011c58727_data_value,0a50e322-460b-4336-bb79-226eda33c269_data_value,b2c7578e-859e-40b3-92ad-bc9513d91925_data_value,e56a0b30-99e3-488e-b91e-16020c38083c_data_value,df23830b-1c17-46e6-a788-65f6e48f77eb_data_value,89315b27-8146-4168-ad03-6a6d636644b7_data_value,7f3aced4-354d-4bc2-9559-e80853bf6085_data_value,6b4a9465-7693-4165-9036-8d32ad501ce2_data_value,e8ab4d4d-0fec-4d89-9257-c7ed5929d664_data_value,dcd5fc5a-f8c4-4b1d-bed2-086f04bc0036_data_value,103f84cd-f7d9-415b-aeaf-8b603ae0a477_data_value,c6709399-0dd1-4747-8963-c0aec6719130_data_value,013835c9-87de-490d-a5b6-861ea10c00ec_data_value,5cbbeb13-560d-4eef-8a25-a101c29ccdcc_data_value,fa64fe48-87d9-4b3c-b09b-98ec2592901d_data_value,f83c090d-27df-47e7-a230-93d1564a528e_data_value,3bda4d64-072c-45a0-b227-5549232ce0ca_data_value,1deac55e-283f-45c7-b539-cddb526358c6_data_value,86bf0a9a-e1e8-4046-a842-32fc5de813c1_data_value,24038d44-e738-4aad-891b-2804dce0fd22_data_value,b8419ce4-42fa-4466-962d-45e284937df5_data_value,f7aa3ce6-4c26-4db9-8fdf-b07cb7536db3_data_value,2457fba4-0238-4c56-9211-e0c930e12c5a_data_value,f7b91092-08fc-4008-a86c-3dbc627939aa_data_value,5ca12a47-a10a-48ad-8310-0ea13ba21bdb_data_value,75d08250-9e63-4a65-a4bf-b018b9ad5d24_data_value,655ec994-9900-4a3d-9077-b118d0c56020_data_value,b10665ad-3ec9-4fd1-9cc1-2f81c848c34e_data_value,11f6759e-b850-47ab-93fd-cdce3c7659f2_data_value,acef7c79-fe54-4aed-91ce-3adf5d9c2d28_data_value,4f4b1dca-d97c-48f9-bc97-239deca351e8_data_value,1bcc6ca3-f9d1-4ace-9f69-3a745a9fbd8e_data_value,085de4ed-e467-4d2e-82a8-34a57ab5035c_data_value,92a83ae1-0dae-492a-b5a2-4b279d3ae982_data_value,263634aa-b26f-4e74-ab85-9a452e559be4_data_value,7263c3da-27de-48fc-b57a-afdd7cbae397_data_value,eb66db47-92a1-4303-9665-f0354fee3c06_data_value,6aed54ad-e99c-4e5a-8029-6fc619fba20a_data_value,7a2ad724-3e88-425e-a4dd-4ebe04978b76_data_value,c002dc02-8683-4c5f-86ae-4cafce743d8c_data_value,0c192b3e-8476-4ff3-8804-9ffacde4ccee_data_value,7ba905a4-ebe3-42eb-850a-de5cfb3b1481_data_value,353bc47f-0f9f-4182-85d8-50f104b38241_data_value,5a52f0a7-c103-4509-a5cd-a5ca05281390_data_value,e47502dc-49a4-4c14-95c5-ee29ffcad112_data_value,4492a537-df3a-41cb-9d7b-6b4c9b5b2cd

In [0]:
### For Dev and Testing, load back the table and view
# lines tested: Borg_Warner_XYZ_L14, Dana_Birmingham_ASC1, Dana_Columbia_Loop_1, Dana_Dry_Ridge_700_line
pivot_load_df = spark.read.format('delta').load(pivotRootDir + "Dana_Dry_Ridge_700_line/")
pivot_load_df.registerTempTable("pivot_results_table")
display(pivot_load_df)

part_number,serial_number,timestamp,station_list,measurement_date,05ed69e8-fc4a-446e-8e80-ae389f54fa47_data_value,0698bcf9-cc1a-48f0-b1f2-73583636ef7e_data_value,06ddc89f-fd31-444f-8beb-26e7203e3cae_data_value,086f24c5-4a59-49eb-84bf-67bc472fbf62_data_value,0e65e3b6-c42c-4e7f-be0e-740ee0fc201f_data_value,0efef437-de99-4ee5-9af5-f3f9198b57cd_data_value,0f5a55dd-18ac-47af-a218-07c9403b2041_data_value,106f2f92-de80-4f75-ba6c-2bd1bd36c1ee_data_value,15ba17dc-025f-4549-bbde-a44a71f826c0_data_value,1c436f7d-750f-49f8-874d-c24469e1e765_data_value,1d842280-eab2-4b77-8d92-0a690958a3f0_data_value,1fe47c11-a899-4069-97f0-f34074e0a705_data_value,23146a96-292c-45f9-b933-eb1ecc47dcd8_data_value,2636dcfe-0638-4e10-b4cb-fa79d7efb78b_data_value,28f4525e-ce1f-4456-a9b2-5dfeb1c52494_data_value,2e85c844-881a-4baa-b68b-7027a4343663_data_value,2f4a8dfc-ef53-4125-b400-d16607517385_data_value,3f478141-eef4-4ecd-b5fd-1ce865145100_data_value,40f5d3a4-d77f-47ec-b642-87cd95005f8f_data_value,501d4f05-a424-49ad-8a49-7ecb414b4f92_data_value,542e1a37-6f35-4b00-8116-41c6ad9efcdf_data_value,5532ee91-86ed-4ffa-a4ca-5cd7704d9659_data_value,5a157d6d-9b9f-489c-b186-1b1c07c29ac3_data_value,65106b69-18b4-471f-964e-2b18a546c3e9_data_value,67c50b1f-77aa-44f2-9b16-394944844e09_data_value,801d8e1b-bf76-4d31-9f24-a39fd93521b6_data_value,8367a8e4-e758-4fc9-93eb-87269d35269b_data_value,83a42fac-14bb-4515-8ba0-2c12c63bbf00_data_value,85452ac8-edca-4365-bde3-783e141da67a_data_value,858a3369-dfda-4239-afdc-d6013c0bc49e_data_value,860492a1-8670-4648-a3ca-aad37c7863c5_data_value,892e820a-a631-4196-844d-57802630fa6c_data_value,8b1c6a11-7b06-4290-a733-430501e91b4a_data_value,8b1e1a1e-5c3d-4731-9735-dc6a053542fa_data_value,8e67a8e9-9a60-41a9-82d9-d874bac28609_data_value,8e960fca-0e92-47ca-b484-1729c3fb93eb_data_value,8f01088b-7a3f-4c57-9265-1efa69f8e42d_data_value,8f560d8a-3304-4e7f-b681-412f11688a1d_data_value,908b3f97-582a-4056-bf24-eb794b9aaff7_data_value,91c94d07-91c9-448c-83b4-6035497fcb76_data_value,9a211258-576c-48e7-a2d4-dc76fb61fd22_data_value,9af2c6a0-cdf8-4972-9b56-edaec13aefab_data_value,9d920fd6-fae0-4593-b204-c5cea6554f8a_data_value,9da9b557-7c3c-4328-988a-f010db164331_data_value,a2c34279-fe87-498c-8129-ded0525a27c2_data_value,aae7af78-49b0-4da2-8d2e-b07a9e0f0a8f_data_value,add3502a-05e8-4880-a268-36e9e49ac89c_data_value,b438a6ec-7647-4413-b22f-9a26c4567e0c_data_value,b9f9243f-f876-44e0-ae06-3b7c08be03db_data_value,bf6a1ce4-ee2e-42c5-b3b8-e156dd75fc95_data_value,da3f7d51-fa8b-4ce9-834f-1476caf5caf3_data_value,dd6fdb45-b3c6-4542-ab7c-294c8531dca8_data_value,e7cd80ca-1ca9-4a9c-97d3-a931eee11550_data_value,e8d257a5-6235-44c4-8753-027bc4423d25_data_value,ebdb5665-54bd-448c-a070-b9015889b2f5_data_value,eeb0323e-7e30-4660-b6d5-f27ce36b6bbb_data_value,f1b0b446-9a8b-41e3-8fc8-154d829b7af9_data_value,fcafc844-5168-447f-aa34-afeb9f2cab72_data_value,fd8f319e-bc5d-497d-9eee-d9ec3ebe113f_data_value,ff3d4ccf-312d-4a4d-a685-bfbdd672bb47_data_value
10139182,WDRK700203100007,2020-11-05 18:31:11.079725+00:00,List(op_120),2020-11-05,null,null,null,4.005,-1,null,null,0,null,0,0,36.24,null,1,null,null,null,0,null,null,null,null,null,null,null,null,null,Pyxis,null,null,2,null,null,0,null,null,null,0,null,null,0,null,0,0,0,1,0,null,0,Pyxis,0,null,null,null,null,null,3,0,null,null
10139182,WDRK700203100007,2020-11-05 18:11:27.154726+00:00,List(op_100),2020-11-05,56,null,null,null,null,null,null,null,1,null,null,null,2.502,null,null,null,null,null,null,null,3,null,42.778,null,null,-1,11.845,null,174.972,null,null,null,Pyxis,null,null,1,null,null,null,null,null,160,null,null,null,null,null,null,null,null,null,null,null,Pyxis,null,null,null,null,2,
10139182,WDRK700203100007,2020-11-05 18:27:14.006752+00:00,List(op_110),2020-11-05,null,335.452,,null,null,4.403,,null,null,null,null,null,null,null,,298,3.152,null,0.001,750,null,2,null,3,7,null,null,null,null,,null,51.252,null,null,4.239,null,0.689,null,0,0,null,null,null,null,null,null,null,2.552,null,null,null,,3.55,null,1,2.202,null,null,null,n

In [0]:
%sql
drop table if exists pivot_results_table;
create table pivot_results_table
using DELTA
location 'dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14/';

OPTIMIZE pivot_results_table ZORDER BY (serial_number);

path,metrics
dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 12, List(minCubeSize(107374182400), List(0, 0), List(12, 79774), 0, List(0, 0), 0, null), 0, 12, 12, false)"


In [0]:
%sql describe history pivot_results_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-11-22T15:21:02.000+0000,1094680775067414,ljin@acerta.ai,MERGE,"Map(predicate -> (((s.`part_number` = u.`part_number`) AND (s.`serial_number` = u.`serial_number`)) AND ((s.`timestamp` = u.`timestamp`) AND (s.`station_list` = u.`station_list`))), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1689534426296516),1110-132154-x22gtc30,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 12, executionTimeMs -> 2665, numTargetRowsInserted -> 297, scanTimeMs -> 363, numTargetRowsUpdated -> 0, numOutputRows -> 297, numTargetChangeFilesAdded -> 0, numSourceRows -> 297, numTargetFilesRemoved -> 0, rewriteTimeMs -> 2279)",null
0,2021-11-22T15:20:51.000+0000,1094680775067414,ljin@acerta.ai,WRITE,"Map(mode -> Append, partitionBy -> [""measurement_date""])",null,List(1689534426296516),1110-132154-x22gtc30,null,SnapshotIsolation,true,"Map(numFiles -> 0, numOutputBytes -> 0, numOutputRows -> 0)",null


In [0]:
### This is the seconda phase of transformation which compute 'run' information into the dataframe.
### this is used by RCA analysis.

from pyspark.sql.functions import col, from_json, to_json, concat, lit, window, when, desc, concat_ws, regexp_replace, collect_set
import pyspark.sql.functions as f
from pyspark.sql.window import Window


#**********************************************************************************************************************
# LOAD PIVOT TABLE
#**********************************************************************************************************************
pivot_load_df = spark.read.format('delta').load(pivotRootDir + "Borg_Warner_XYZ_L14/")


#**********************************************************************************************************************
# READ FEATURES LIST FROM SCHEMA TABLE
#**********************************************************************************************************************
dataSourceId, schemaVersion = '0.1.0', '0.1.1'
pivot_feature_cols = getFeatureListLocal(dataSourceId)


#**********************************************************************************************************************
# DEFINE WINDOWS FOR AGGREGATION
#**********************************************************************************************************************

w1 = Window.partitionBy(["part_number", "serial_number"]).orderBy("timestamp")
w2 = Window.partitionBy(["part_number", "serial_number", "station"]).orderBy("timestamp")
w3 = Window.partitionBy(["part_number", "serial_number", "station", "consecutive_idx"]).orderBy(f.desc("timestamp"))
w4 = Window.partitionBy(["part_number", "serial_number"]).rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
w5 = Window.partitionBy(["part_number", "serial_number", "target_operation"]).orderBy("target_test_date")


#**********************************************************************************************************************
# TRANSFORM STEP: add derived columns to prepare for path extraction
#
# New column details:
# 1. op_order: order of operations for a part goes through the assembly line, starting from 1 for the first operation
# 2. op_rerun: # of times a part passes a station
# 3. consecutive_idx (TEMPORARY): consecutive visits of a part to the same station (i.e. no other station in between) have the same consecutive_idx
# 4. is_last_measure: if a part is consecutively measured at a station, only the last measurement have is_last_measure = 1
#**********************************************************************************************************************

run_order_df = (pivot_load_df
                .withColumn("station", f.element_at("station_list", 1))
                .withColumn("op_order", f.dense_rank().over(w1))
                .withColumn("op_rerun", f.dense_rank().over(w2))
                .withColumn("consecutive_idx", col("op_order")-col("op_rerun"))
                .withColumn("is_last_measure", f.dense_rank().over(w3))
                .select("part_number", "serial_number", "station", "timestamp", "op_order", "op_rerun", "is_last_measure", *pivot_feature_cols)
               )


#**********************************************************************************************************************
# TRANSFORM STEP: identify 'path'(s) for each unit
#
# OUTPUT:  a dataframe with one row for each 'path', identified by 
#          1) the part and serial number of the assembled unit (part_number, serial_number)
#          2) the target station and the corresponding test date (target_operation, target_test_date)
#          3) a pass (1)/fail (0) flag indicating whether the part passed or failed at the target station (pass_fail)
#          4) a run number indicating the number of attempts for the unit to reach the target station
#**********************************************************************************************************************

target_ops_df = (run_order_df.withColumn("max_op_order", f.max("op_order").over(w4))
                      .filter((col("station")==target) & (col("is_last_measure")==1))
                      .withColumn("max_target_rerun", f.max("op_rerun").over(w4))
                      .withColumn("pass_fail", f.when(col("op_order")==col("max_op_order"), 2)
                                                 .when(col("op_rerun")==col("max_target_rerun"), 1)
                                                 .when(col("op_rerun")<col("max_target_rerun"), 0)
                                                 .otherwise(-1)
                                 )
                      .select("part_number", "serial_number", "station", "timestamp", "pass_fail")
                      .withColumnRenamed("timestamp", "target_test_date")
                      .withColumnRenamed("station", "target_operation")
                      .withColumn("run_number", f.row_number().over(w5))
                     )


#**********************************************************************************************************************
# TRANSFORM STEP: associate the pivotted sensor measurements to their corresponding path
#**********************************************************************************************************************

target_ops_df.registerTempTable("target_ops")
run_order_df.registerTempTable("run_order")
join_target_with_ops_sql = f"""
select o.part_number as part_number, o.serial_number as serial_number, run_number, 
    pass_fail, station, timestamp, `{'`,`'.join(pivot_feature_cols)}`,
	dense_rank() over (
		partition by o.part_number, o.serial_number, target_test_date, station order by timestamp
	) as op_rerun_in_path
from target_ops o join run_order r 
on o.part_number=r.part_number and o.serial_number=r.serial_number and target_test_date>=timestamp 
"""
cube_run_df= spark.sql(join_target_with_ops_sql)



#**********************************************************************************************************************
# WRITE CUBE TO DISK
# Write cube dataframe to parquet on disk. This completes the cube transformation.
#**********************************************************************************************************************

saveCube = cube_run_df.write.mode('append').parquet(cubeRunRootDir + "Borg_Warner_XYZ_L14/")  

display(cube_run_df)

part_number,serial_number,run_number,pass_fail,station,timestamp,b8d46933-5220-4bad-8221-c54715a835fd_Torque,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque,op_rerun_in_path
12639700098,0482106351600004,1,2,L14_Core_03,2021-03-16 04:08:46.070,null,null,null,null,90,null,null,null,null,143.05,null,null,null,null,1
12639700098,0482106351600004,1,2,L14_Final_03,2021-03-16 04:15:22.973,null,null,10.45,null,null,34,null,null,null,null,null,null,null,null,1
12639700098,0482106351600004,1,2,L14_Final_03,2021-03-16 04:15:55.177,null,null,10.48,null,null,49,null,null,null,null,null,null,null,null,2
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:06.337,null,null,null,null,null,null,null,null,null,null,null,988,null,17.04,1
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:26.523,null,56,null,null,null,null,13.69,null,null,null,null,null,null,null,2
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:31.570,null,62,null,null,null,null,13.64,null,null,null,null,null,null,null,3
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:37.837,null,80,null,null,null,null,13.73,null,null,null,null,null,null,null,4
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:45.057,null,69,null,null,null,null,13.75,null,null,null,null,null,null,null,5
12639700098,0482106452300007,1,2,L14_Final_04,2021-03-16 04:10:50.377,null,null,null,null,null,null,null,null,null,null,null,992,null,17.08,1
12639700098,0482106452300007,1,2,L14_Final_04,2021-03-16 04:11:11.290,null,55,null,null,null,null,13.78,null,null,null,null,null,null,null,2
